In [1]:
!pip install -qU transformers langchain datasets chromadb langchain-huggingface


  You can safely remove it manually.


In [3]:
!pip install -qU "pydantic>=2.7.4"
!pip install -qU langsmith==0.1.125
!pip install -qU langchain-huggingface chromadb==0.4.22 datasets==2.16.1


In [4]:
import pandas as pd
from datasets import load_dataset
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from transformers import pipeline


In [25]:
# Load the MedQuad Medical QnA Dataset from Hugging Face and convert to a pandas DataFrame.
data = load_dataset("keivalya/MedQuad-MedicalQnADataset", split='train')
data = data.to_pandas()

# Optional: Limit the dataset size for faster execution (e.g., first 100 records)
# data = data[0:100]

# Display a preview of the dataset.
print("Dataset preview:")
print(data.head())


Dataset preview:
             qtype                                           Question  \
0   susceptibility  Who is at risk for Lymphocytic Choriomeningiti...   
1         symptoms  What are the symptoms of Lymphocytic Choriomen...   
2   susceptibility  Who is at risk for Lymphocytic Choriomeningiti...   
3  exams and tests  How to diagnose Lymphocytic Choriomeningitis (...   
4        treatment  What are the treatments for Lymphocytic Chorio...   

                                              Answer  
0  LCMV infections can occur after exposure to fr...  
1  LCMV is most commonly recognized as causing ne...  
2  Individuals of all ages who come into contact ...  
3  During the first phase of the disease, the mos...  
4  Aseptic meningitis, encephalitis, or meningoen...  


In [26]:
# Create a document loader using the 'Answer' column as the page content.
df_loader = DataFrameLoader(data, page_content_column="Answer")
df_document = df_loader.load()

# Optionally, display the first two documents.
print("First two documents:")
print(df_document[:2])

from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,         # Target maximum chunk size in characters
    chunk_overlap=50,       # Overlap between consecutive chunks
    separators=["\n\n", "\n", " ", ""]  # Try splitting on double newline, newline, space, then character
)
texts = recursive_splitter.split_documents(df_document)

# Optionally, check the lengths of some chunks
for i, chunk in enumerate(texts[:5]):
    print(f"Chunk {i} length: {len(chunk.page_content)}")



First two documents:
[Document(metadata={'qtype': 'susceptibility', 'Question': 'Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?'}, page_content='LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents.  Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent. Person-to-person transmission has not been reported, with the exception of vertical transmission from infected mother to fetus, and rarely, through organ transplantation.'), Document(metadata={'qtype': 'symptoms', 'Question': 'What are the symptoms of Lymphocytic Choriomeningitis (LCM) ?'}, page_content='LCMV is most commonly recognized as causing neurological disease, as its name implies, though infection without symptoms or mild febrile illnesses are more common clinical manifestations. \n                \nFor infected persons who do become

In [ ]:
# # Initialize Hugging Face Embeddings using an open source model.
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# # Create a Chroma vector store from the documents using these embeddings.
# # Adjust 'persist_directory' as needed.
# directory_cdb = "./chromadb"
# chroma_db = Chroma.from_documents(df_document, embeddings, persist_directory=directory_cdb)


In [27]:
import shutil
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Define a new persistent directory to avoid conflicts
directory_cdb = "./chromadb_temp"
shutil.rmtree(directory_cdb, ignore_errors=True)  # Remove if exists

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Create a new Chroma vector store in the fresh directory
chroma_db = Chroma.from_documents(df_document, embeddings, persist_directory=directory_cdb)



KeyboardInterrupt



In [23]:
import os
# Optionally disable the symlink warning on Windows
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline

# Create a Hugging Face Transformers pipeline using an open source text generation model.
# Using "google/flan-t5-small" which works on CPU.
pipe = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    tokenizer="google/flan-t5-small",
    framework="pt",  # PyTorch backend
    device=-1        # Use CPU (set device to 0 for GPU if available)
)

# Wrap the pipeline in a LangChain LLM interface.
llm = HuggingFacePipeline(pipeline=pipe)

# You can now use `llm` in your Retrieval QA chain or elsewhere.
print("LLM initialized. Device:", pipe.device)


Device set to use cpu


LLM initialized. Device: cpu


In [ ]:
# # Create a Hugging Face Transformers pipeline using an open source text generation model.
# # Here we use "google/flan-t5-small" which is lightweight and works on CPU.
# pipe = pipeline(
#     "text2text-generation",
#     model="google/flan-t5-small",
#     tokenizer="google/flan-t5-small",
#     framework="pt",  # PyTorch backend
#     device=-1        # Use CPU (set device to 0 for GPU if available)
# )

# # Wrap the pipeline in a LangChain LLM interface.
# llm = HuggingFacePipeline(pipeline=pipe)


In [34]:
from langchain.prompts import PromptTemplate

# Define a custom map prompt to summarize each document context with a focus on LCM symptoms.
map_prompt = PromptTemplate(
    input_variables=["text"],
    template=(
        "You are a knowledgeable medical expert. Summarize the following context, focusing on the symptoms of LCM. "
        "Include details if the text mentions a biphasic febrile illness with an initial phase that includes symptoms "
        "such as fever, malaise, lack of appetite, muscle aches, headache, nausea, and vomiting. \n\nContext:\n{text}\n\nSummary:"
    )
)

# Define a custom combine prompt to merge the summaries into a detailed answer.
combine_prompt = PromptTemplate(
    input_variables=["summaries", "question"],
    template=(
        "You are a knowledgeable medical expert. Combine the following summaries to answer the question in detail. "
        "Ensure you state that the main symptom of LCM is a biphasic febrile illness that includes symptoms such as fever, "
        "malaise, lack of appetite, muscle aches, headache, nausea, and vomiting if supported by the context. \n\n"
        "Question: {question}\n\nSummaries:\n{summaries}\n\nAnswer:"
    )
)

# Optionally adjust the retriever to limit the number of retrieved documents.
retriever = chroma_db.as_retriever(search_kwargs={"k": 1})

# Build the Retrieval QA chain using the map_reduce chain type with custom prompts.
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",  # This chain summarizes and then combines info.
    retriever=retriever,
    chain_type_kwargs={
         "map_prompt": map_prompt,
         "combine_prompt": combine_prompt
    },
    verbose=True
)

# Test the retrieval chain by asking the question.
query = "What is the main symptom of LCM?"
result = qa.run(query)
print("Query:", query)
print("QA Output:", result)


ValidationError: 1 validation error for MapReduceDocumentsChain
map_prompt
  Extra inputs are not permitted [type=extra_forbidden, input_value=PromptTemplate(input_vari...t:\n{text}\n\nSummary:'), input_type=PromptTemplate]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden

In [32]:
# Create conversation memory to store a short chat history (optional).
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=4,  # Number of past messages stored in memory
    return_messages=True
)

# Build the Retrieval QA chain using the LLM and the vector store retriever.
retriever = chroma_db.as_retriever(search_kwargs={"k": 2})
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce", #stuff
    retriever=retriever
)




In [35]:
# Test the retrieval chain by asking a question.
query = "How to diagnose Lymphocytic Choriomeningitis?"
result = qa.run(query)
print("Query:", query)
print("QA Output:", result)

Query: How to diagnose Lymphocytic Choriomeningitis?
QA Output: You should be able to diagnose it.
